## Import Dependencies

In [1]:
import csv
from pathlib import Path
import pandas as pd
import glob
from datetime import datetime, timedelta
import numpy as np
# from sklearn import preprocessing
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression, RidgeCV
# from sklearn.metrics import mean_squared_error
import seaborn as sns
import pandas_profiling as pp
from settings import *
# from settings import DATA_DIR

In [2]:
download_dir = 'D:\Performance_All'
download_dir

'D:\\Performance_All'

In [3]:
DATA_DIR

'data'

In [4]:
outputFileName = 'FMPerfProcessedALL.csv'
outpath = "/".join([DATA_DIR,outputFileName])
outpath

'data/FMPerfProcessedALL.csv'

# Get the Single-Family Loan Performance Data Files from Fannie Mae

From the Fannie Mae [website:](https://loanperformancedata.fanniemae.com/lppub/index.html#Single-Family_Loan_Performance_Data_Files) we downloaded the Performance dataset (26GB zip file)

The [Performance file layout](https://loanperformancedata.fanniemae.com/lppub-docs/FNMA_SF_Loan_Performance_File_layout.pdf) was as follows:
![Performance File Layout](images/PerformanceFileLayout.jpg)

In [5]:
# Create mapping for the header of the files based on above file format
allheaderline = ['id','rptPeriod','sellerName','currIntRate','currUPB','loanAge','monMatur','adjMonMatur',\
              'maturDate','mSA','deliqStatus','modFlag','zeroBalCode','zeroBalDate','lastPdInstDate',\
              'forecloDate','dispDate','forecloCost','propRepCost','recovCosts','miscCost','holdTaxCost',\
              'saleProceed','credEnhProceed','repurchProceed','otherForecloProceed','nonIntUPB',\
              'prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator']
colchoices = [0,9,10,12]
headerline = ['id','mSA','deliqStatus','zeroBalCode']

In [6]:
# Loop through each file from the unzipped download file - which gave a set of 76 individual files

#Create an empty array to hold dataframes to later concatonate
li = []

#Loop through every file in the directory and append into array
for in_path in Path(download_dir).glob('Performance*.txt'):
    df = pd.read_csv(in_path, sep="|", index_col=None, header=None, usecols=colchoices)
    df.columns=headerline
    df = df.set_index('id')
    # Isolate the deliqStatus column to speed up aggregation calculations
    deliq = df[['deliqStatus']].copy()
    # deliqStatus has one value of 'X' amongst integer values and NaN values
    # First replace 'X' values with NaN
    deliq['deliqStatus'] = deliq['deliqStatus'].replace('X',np.nan).astype(float)
    # Determine the number of zero (0) values representing successful payments for a loan
    deliqGood = deliq[deliq['deliqStatus'] == 0].groupby('id').count()
    # Determine the number of values > 0 representing delinquent payments for a loan
    deliqBad = deliq[deliq['deliqStatus'] > 0].groupby('id').count()
    # Determine the max() of deliqStatus representing highest numberdelinquent payments in a row for a loan
    deliqMax = deliq[deliq['deliqStatus'] > 0].groupby('id').max()
#     # Select only rows that have a zeroBalCode with a value ## Commented out to include ALL records
#     zbc_df = df[df['zeroBalCode'].notnull()]
    df = df.drop(columns=['deliqStatus'])
    #Merge the zbc_df with the deliqGood
    zbc_merge = pd.merge(df, deliqGood, on=['id'])
    #Merge the zbc_df with the deliqBad
    zbc_merge = pd.merge(zbc_merge, deliqBad, on=['id'])
    #Merge the zbc_df with the deliqMax
    zbc_merge = pd.merge(zbc_merge, deliqMax, on=['id'])
    # Rename all the deqliq* columns
    zbc_merge = zbc_merge.rename(columns={'deliqStatus_x':'deliqGood','deliqStatus_y':'deliqBad','deliqStatus':'deliqMax'})
    # Add the zbc_merge df to the li list to be able to concat them later
    li.append(zbc_merge)

In [7]:
len(li)

78

In [8]:
#Concatonate all the array elements into one giant dataframe
processed_df = pd.concat(li, axis=0)
#df.columns = allheaderline

In [9]:
processed_df.shape

(443356941, 5)

In [10]:
processed_df.head()

,mSA,zeroBalCode,deliqGood,deliqBad,deliqMax
id,,,,,
100021703104,35980,NaN,33,1,1.0
100021703104,35980,NaN,33,1,1.0
100021703104,35980,NaN,33,1,1.0
100021703104,35980,NaN,33,1,1.0
100021703104,35980,NaN,33,1,1.0


In [11]:
#Review the columns for datatypes and how many non-null values 
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443356941 entries, 100021703104 to 999888644871
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   mSA          int64  
 1   zeroBalCode  float64
 2   deliqGood    int64  
 3   deliqBad     int64  
 4   deliqMax     float64
dtypes: float64(2), int64(3)
memory usage: 19.8 GB


### Convert the origDate,firstPmtDate columns from strings to datetime datatypes
Since this will be such a large dataset I tested a number of methods to change datatypes

In [12]:
processed_df.shape

(443356941, 5)

In [13]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443356941 entries, 100021703104 to 999888644871
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   mSA          int64  
 1   zeroBalCode  float64
 2   deliqGood    int64  
 3   deliqBad     int64  
 4   deliqMax     float64
dtypes: float64(2), int64(3)
memory usage: 19.8 GB


In [14]:
processed_df.reset_index(inplace=True)


In [15]:
df = processed_df.copy()

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443356941 entries, 0 to 443356940
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   mSA          int64  
 2   zeroBalCode  float64
 3   deliqGood    int64  
 4   deliqBad     int64  
 5   deliqMax     float64
dtypes: float64(2), int64(4)
memory usage: 19.8 GB


In [17]:
df.head(50)

,id,mSA,zeroBalCode,deliqGood,deliqBad,deliqMax
0,100021703104,35980,NaN,33,1,1.0
1,100021703104,35980,NaN,33,1,1.0
2,100021703104,35980,NaN,33,1,1.0
3,100021703104,35980,NaN,33,1,1.0
4,100021703104,35980,NaN,33,1,1.0
5,100021703104,35980,NaN,33,1,1.0
6,100021703104,35980,NaN,33,1,1.0
7,100021703104,35980,NaN,33,1,1.0
8,100021703104,35980,NaN,33,1,1.0
9,100021703104,35980,NaN,33,1,1.0


In [18]:
df.id.value_counts()

560090795128    240
467514578232    240
686999233251    240
920588411361    240
211348381306    240
               ... 
496649694956      3
815712163594      3
296454967408      3
521625267015      3
586616167886      3
Name: id, Length: 5485661, dtype: int64

In [19]:
df.drop_duplicates( subset='id',keep='last', inplace=True)

In [27]:
df['zeroBalCode'].fillna(0, inplace=True)

In [28]:
df['zeroBalCode'] = processed_df['zeroBalCode'].astype(int)

In [29]:
df['deliqMax'] = processed_df['deliqMax'].astype(int)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5485661 entries, 34 to 443356940
Data columns (total 6 columns):
 #   Column       Dtype
---  ------       -----
 0   id           int64
 1   mSA          int64
 2   zeroBalCode  int32
 3   deliqGood    int64
 4   deliqBad     int64
 5   deliqMax     int32
dtypes: int32(2), int64(4)
memory usage: 251.1 MB


# Write the result df to csv

In [31]:
df.to_csv(outpath)

In [32]:
df.shape

(5485661, 6)

In [33]:
df.head()

,id,mSA,zeroBalCode,deliqGood,deliqBad,deliqMax
34,100021703104,35980,1,33,1,1
66,100023274028,12060,1,20,11,4
108,100033892603,33100,1,40,1,1
132,100048724218,13460,1,22,1,1
147,100057706656,17140,6,13,1,1
